<img src='http://hilpisch.com/taim_logo.png' width="350px" align="right">

# Financial Theory

**Illustrated based on Numerical Examples**

Dr Yves J Hilpisch | The AI Machine

http://aimachine.io | http://twitter.com/dyjh

## Imports

In [ ]:
!git clone https://github.com/tpq-classes/ai_in_finance.git
import sys
sys.path.append('ai_in_finance')


In [ ]:
import math
import numpy as np
import pandas as pd
from pylab import plt
plt.style.use('seaborn-v0_8')
%matplotlib inline

## Arbitrage Pricing

### By Replication

In [ ]:
P = np.array((0.5, 0.5))

In [ ]:
S0 = 10
S = np.array((20, 5))

In [ ]:
B0 = 10
B = np.array((11, 11))

In [ ]:
def ret(x, x0, P):
    r = np.dot(x, P) / x0 - 1
    return r.round(3)

In [ ]:
ret(S, S0, P)

In [ ]:
ret(B, B0, P)

In [ ]:
M = np.array((S, B)).T
M

In [ ]:
M0 = np.array((S0, B0))

In [ ]:
K = 15

In [ ]:
C = np.maximum(S - K, 0)
C

In [ ]:
phi = np.linalg.solve(M, C)
phi

In [ ]:
np.dot(M, phi).round(2)

In [ ]:
C0 = np.dot(M0, phi)
C0

In [ ]:
ret(C, C0, P)

### By Martingale Measures

In [ ]:
r = ret(B, B0, P)
r

In [ ]:
def ES(Q):
    return np.dot(S, Q) / (1 + r)

In [ ]:
ES(P)

In [ ]:
ES(P) - S0

In [ ]:
def OF(Q):
    return abs(ES(Q) - S0)

In [ ]:
OF(P)

In [ ]:
from scipy.optimize import minimize

In [ ]:
bnds = 2 * [(0, 1)]

In [ ]:
cons = {'type': 'eq', 'fun': lambda Q: Q.sum() - 1}

In [ ]:
opt = minimize(OF, 2 * [1 / 2], bounds=bnds, constraints=cons)
opt

In [ ]:
Q = opt['x'].round(3)
Q

In [ ]:
ret(S, S0, Q)

In [ ]:
ret(B, B0, Q)

In [ ]:
C0 = np.dot(C, Q) / (1 + r)
C0

In [ ]:
ret(C, C0, Q)

## Expected Utility Theory

### Utility Function

Varian (2010) introduces the concept of _utility_ and _utility functions_ as follows:

> In Victorian days, philosophers and economists talked blithely of “utility”
as an indicator of a person’s overall well-being. Utility was thought of as
a numeric measure of a person’s happiness. Given this idea, it was natural
to think of consumers making choices so as to maximize their utility, that
is, to make themselves as happy as possible.

> A utility function is a way of assigning a number to every possible
consumption bundle such that more-preferred bundles get assigned larger
numbers than less-preferred bundles.

In [ ]:
def u(c):
    return np.sqrt(c)

In [ ]:
def du(c):
    return 0.5 / np.sqrt(c)

In [ ]:
x = np.linspace(0.01, 3, 250)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(x, u(x), label='$u(c)$')
plt.plot(x, du(x), label='$du(c)$')
plt.xlabel('consumption or $c$')
plt.ylabel('utility or $u(c)$')
plt.legend();

### Expected Utility

Varian (2010) writes:

>  Thus the expression $\pi_1 u(c_1) + \pi_2u(c_2)$
represents the average utility, or the expected utility, of the pattern of
consumption $(c_1, c_2)$.

Above, $\pi_s$ is the probabilty for state $s$ to unfold with $\pi_1 + \pi_2 = 1$. Accordingly, $c_s$ is consumption (wealth) in state $s$.

>For this reason, we refer to a utility function with the particular form
described here as an expected utility function, or, sometimes, a von
Neumann-Morgenstern utility function.

In [ ]:
def Eu(M, phi, P):
    w = np.dot(M, phi)
    return np.dot(u(w), P)

In [ ]:
w0 = 10

In [ ]:
Eu(M, (1.0, 0.0), P)

In [ ]:
Eu(M, (0.75, 0.25), P)

In [ ]:
Eu(M, (0.5, 0.5), P)

In [ ]:
Eu(M, (0.25, 0.75), P)

In [ ]:
Eu(M, (0.0, 1.0), P)

In [ ]:
bnds = 2 * [(0, np.inf)]
bnds

In [ ]:
cons = {'type': 'eq', 'fun': lambda phi: np.dot(M0, phi) - w0}

In [ ]:
opt = minimize(lambda phi: -Eu(M, phi, P), (2, 0.5),
               bounds=bnds, constraints=cons)
opt

In [ ]:
phi = opt['x']
phi.round(3)

In [ ]:
Eu(M, phi, P)

## Representative Agent

... or equilibrium pricing

In [ ]:
MS = np.array(((1, 0), (0, 1))).T
MS

In [ ]:
w0 = 1 / 1.1
w0

In [ ]:
bnds = 2 * [(0, np.inf)]

In [ ]:
cons = {'type': 'eq', 'fun': lambda p: np.sum(p) - w0}

In [ ]:
opt = minimize(lambda p: -Eu(MS, (1, 1), P), (1, 1),
               bounds=bnds, constraints=cons)
opt

In [ ]:
p = opt['x']
p

In [ ]:
S0 = np.dot(S, p)  # equilibrium pricing of the stock
S0

In [ ]:
ret(S, S0, P)

In [ ]:
B0 = np.dot(B, p)
B0

In [ ]:
ret(B, B0, P)

In [ ]:
M0 = np.array((S0, B0))

In [ ]:
phi = np.linalg.solve(M, C)

In [ ]:
C0 = np.dot(M0, phi)
C0

In [ ]:
np.dot(C, p)

In [ ]:
ret(C, C0, P)

## Modern Portfolio Theory (MPT)

Sharpe (1964) points out:

> Markowitz [(1952, 1959)] following Von Neumann and Morgenstern, developed an analysis based on the expected utility maxim and proposed a general solution for the portfolio selection problem.

Markowitz (1952) postulates:

> We next consider the rule that the investor does (or should) consider expected return a desirable thing and variance of return an undesirable thing. This rule has many sound points, both as a maxim for, and hypothesis about, investment behavior.

### Financial Data

In [ ]:
url = 'http://hilpisch.com/tr_eikon_eod_data.csv'

In [ ]:
raw = pd.read_csv(url, index_col=0, parse_dates=True).dropna()

In [ ]:
raw.info()

In [ ]:
symbols = ['AAPL.O', 'MSFT.O', 'INTC.O']

In [ ]:
data = raw[symbols].copy()

In [ ]:
rets = np.log(data / data.shift(1)).dropna()

In [ ]:
rets.mean()

In [ ]:
rets.mean() * 252

In [ ]:
rets.std()

In [ ]:
rets.std() * math.sqrt(252)

### Portfolio Statistics

In [ ]:
w = len(symbols) * [1 / len(symbols)]
w

In [ ]:
np.dot(rets.mean(), w) * 252

In [ ]:
def port_return(w):
    return np.dot(rets.mean(), w) * 252

In [ ]:
port_return(w)

In [ ]:
rets.cov()

In [ ]:
np.dot(w, np.dot(rets.cov(), w)) * 252  # portfolio variance

In [ ]:
math.sqrt(np.dot(w, np.dot(rets.cov(), w)) * 252)  # portfolio volatility

In [ ]:
def port_volatility(w):
    return math.sqrt(np.dot(w, np.dot(rets.cov(), w)) * 252)

In [ ]:
port_volatility(w)

### Simulation

In [ ]:
N = 500

In [ ]:
rw = np.random.random((N, len(symbols)))

In [ ]:
rw = (rw.T / rw.sum(axis=1)).T

In [ ]:
rw[:5]

In [ ]:
pv = [(port_volatility(w), port_return(w))
     for w in rw]

In [ ]:
pv = np.array(pv)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(pv[:, 0], pv[:, 1], 'ro')
plt.ylabel('portfolio return')
plt.xlabel('portfolio risk');

### Minimum Risk Portfolio

In [ ]:
bnds = len(symbols) * [(0, 1)]
bnds

In [ ]:
cons = {'type': 'eq', 'fun': lambda w: w.sum() - 1}

In [ ]:
opt = minimize(port_volatility, w, bounds=bnds, constraints=cons)
opt

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(pv[:, 0], pv[:, 1], 'r.',
         label='random portfolios')
plt.ylabel('portfolio return')
plt.xlabel('portfolio risk')
plt.plot(port_volatility(opt['x']),
         port_return(opt['x']), 'yo',
         ms=10, label='minimum risk portfolio')
plt.legend();

### Empirical Analyses

Modern Portfoluo Theory (MPT) assumes that investors only care about **the first and second moment of the return distribution**.

Only the **(log-)normal distribution** is fully characterized by its first moment (_expectation_) and second moment (_standard deviation_).

#### An Example

In [ ]:
N = 1000

In [ ]:
snrn = np.random.standard_normal(N)
snrn -= snrn.mean()
snrn /= snrn.std()

In [ ]:
round(snrn.mean(), 4)

In [ ]:
round(snrn.std(), 4)

In [ ]:
plt.hist(snrn, bins=35);

In [ ]:
numbers = np.ones(N) * 1.5
split = int(0.25 * N)
numbers[split:3 * split] = -1
numbers[3 * split:4 * split] = 0
numbers -= numbers.mean()
numbers /= numbers.std()

In [ ]:
round(numbers.mean(), 4)

In [ ]:
round(numbers.std(), 4)

In [ ]:
plt.hist(numbers, bins=35);

### Checking for Normality

In [ ]:
# !conda install -y statsmodels

In [ ]:
import scipy.stats as scs
import statsmodels.api as sm

In [ ]:
def dN(x, mu, sigma):
    ''' Probability density function of a normal random variable x.
    '''
    z = (x - mu) / sigma
    pdf = np.exp(-0.5 * z ** 2) / math.sqrt(2 * math.pi * sigma ** 2)
    return pdf

In [ ]:
def return_histogram(rets):
    ''' Plots a histogram of the returns.
    '''
    plt.figure(figsize=(10, 6))
    x = np.linspace(min(rets), max(rets), 100)
    plt.hist(np.array(rets), bins=50, density=True)
    y = dN(x, np.mean(rets), np.std(rets))
    plt.plot(x, y, linewidth=2)
    plt.xlabel('log returns')
    plt.ylabel('frequency/probability')
    plt.grid(True)

In [ ]:
return_histogram(snrn)

In [ ]:
return_histogram(numbers)

In [ ]:
def return_qqplot(rets):
    ''' Generates a Q-Q plot of the returns.
    '''
    sm.qqplot(rets, line='s')
    plt.xlabel('theoretical quantiles')
    plt.ylabel('sample quantiles')

In [ ]:
return_qqplot(snrn);

In [ ]:
return_qqplot(numbers)

In [ ]:
def print_statistics(rets):
    print("RETURN SAMPLE STATISTICS")
    print("---------------------------------------------")
    print("Skew of Sample Log Returns %9.6f" % scs.skew(rets))
    print("Skew Normal Test p-value   %9.6f" % scs.skewtest(rets)[1])
    print("---------------------------------------------")
    print("Kurt of Sample Log Returns %9.6f" % scs.kurtosis(rets))
    print("Kurt Normal Test p-value   %9.6f" % \
                scs.kurtosistest(rets)[1])
    print("---------------------------------------------")
    print("Normal Test p-value        %9.6f" % \
                scs.normaltest(rets)[1])
    print("---------------------------------------------")

In [ ]:
print_statistics(snrn)

In [ ]:
print_statistics(numbers)

### Real-World Distributions

#### Single Instruments

In [ ]:
symbols = ['.SPX', 'EUR=', 'GLD']

In [ ]:
rets = np.log(raw[symbols] / raw[symbols].shift(1)).dropna()

In [ ]:
symbols

In [ ]:
for sym in symbols:
    return_histogram(rets[sym].values)

In [ ]:
for sym in symbols:
    return_qqplot(rets[sym].values)

In [ ]:
for sym in symbols:
    print('\n{}'.format(sym))
    print(45 * '=')
    print_statistics(rets[sym].values)

#### Portfolio

In [ ]:
for weights in [w, opt['x']]:
    return_histogram(np.dot(rets, weights))

In [ ]:
for weights in [w, opt['x']]:
    return_qqplot(np.dot(rets, weights))

In [ ]:
for weights in [w, opt['x']]:
    print()
    print_statistics(np.dot(rets, weights))

## Capital Asset Pricing Model (CAPM)

Sharpe (1964) assumes the following:

> In order to derive conditions for equilibriumin the capital market we invoke two assumptions. First, we assume a common pure rate of interest, with all investors able to borrow or lend funds on equal terms. Second, weassume homogeneity of investor expectations: investors are assumed to agree on the prospects of various investments &mdash; the expected values, standard deviations and correlation coefficients ... Needless to say, these are highly restrictive and undoubtedly unrealistic assumptions.

According to the CAPM, the expected return for a stock (financial asset) is given by the following relationship:

$$\mu_S = r + \beta \cdot (\mu_M - r)$$

Here, $r$ is the riskless short rate, $\beta_S$ is a stock-specific factor to be determined through statistical methods and $\mu_M$ is the expected return of the market portfolio. $\beta_S$ is given by

$$\beta = \frac{\sigma_{SM}}{\sigma^2_M}$$

with $\sigma_{SM}$ as the covariance between the stock and the market portfolio and $\sigma^2_M$ the variance of the market's returns.

In [ ]:
symbols = ['AAPL.O', '.SPX']
market = '.SPX'
data = raw[symbols]
rets = np.log(data / data.shift(1)).dropna()

In [ ]:
rets.plot(kind='scatter', x=market, y=symbols[0], figsize=(10, 6));

### Formualic Approach

In [ ]:
r = 0.01

In [ ]:
rets.cov() * 252

In [ ]:
cov = rets.cov().iloc[0, -1] * 252
cov

In [ ]:
rets.var() * 252

In [ ]:
var = rets[market].var() * 252
var

In [ ]:
muM = rets[market].mean() * 252
muM

In [ ]:
beta = cov / var
beta

In [ ]:
muS = r + beta * (muM - r)
muS

In [ ]:
rets[symbols[0]].mean() * 252

In [ ]:
rets = np.log(raw / raw.shift(1)).dropna()

In [ ]:
symbols = raw.columns[:7]
symbols

In [ ]:
for sym in raw.columns[:5]:
    print('\n' + sym)
    print(45 * '=')
    cov = rets.cov().loc[sym, '.SPX'] 
    var = rets['.SPX'].var()
    beta = cov / var
    muS = r + beta * (muM - r)
    mean =  rets[sym].mean() * 252
    print('beta: {:.3f} | mu: {:.3f} | mean: {:.3f}'.format(beta, muS, mean))

### Regression Approach 

In [ ]:
np.set_printoptions(suppress=True)

In [ ]:
reg = np.polyfit(rets[symbols[-1]], rets[symbols[0]], deg=1)
reg.round(4)

In [ ]:
reg = np.polyfit(rets[symbols[-1]] - r, rets[symbols[0]] - r, deg=1)
reg.round(4)

In [ ]:
rets.plot(kind='scatter', x=symbols[-1],
          y=symbols[0], figsize=(10, 6))
plt.plot(rets[symbols[-1]], np.polyval(reg, rets[symbols[-1]]), 'r');

In [ ]:
beta = reg[0]
beta

In [ ]:
muS = r + beta * (muM - r)
muS

In [ ]:
rets[symbols[0]].mean() * 252

In [ ]:
res = pd.DataFrame()
for sym in raw.columns[:5]:
    print('\n' + sym)
    print(45 * '=')
    beta = np.polyfit(rets['.SPX'],
                      rets[sym], deg=1)[0]
    muS = muS = r + beta * (muM - r)
    mean =  rets[sym].mean() * 252
    print('beta: {:.3f} | mu: {:.3f} | mean: {:.3f}'.format(beta, muS, mean))
    res = pd.concat((res, pd.DataFrame({'sym': sym, 'beta': beta,
                                   'mu': muS, 'mean': mean}, index=[0])))

In [ ]:
res

In [ ]:
ax = res.round(3).set_index('sym').plot(kind='bar', figsize=(10, 6),
                                  secondary_y='beta')
ax.get_legend().set_bbox_to_anchor((0.2, 1));

In [ ]:
res[['beta', 'mean']].plot(kind='scatter', x='beta',
                         y='mean', figsize=(10, 6))
x = np.linspace(res['beta'].min(), res['beta'].max())
reg = np.polyfit(res['beta'], res['mean'], deg=1)
plt.plot(x, np.polyval(reg, x), 'g--', label='regression')
plt.legend();

In [ ]:
pr = False

In [ ]:
res = pd.DataFrame()
years = range(2010, 2019)
for sym in raw.columns[:5]:
    for year in years:
        sel = rets[(rets.index >= '01-01-{}'.format(year)) &
                  (rets.index <= '31-12-{}'.format(year))]
        
        beta = np.polyfit(sel[market],
                          sel[sym], deg=1)[0]
        muM = sel[market].mean() * 252
        muS = r + beta * (muM - r)
        mean =  sel[sym].mean() * 252
        if pr:
            print('\n' + sym + '| {}'.format(year))
            print(45 * '=')
            print('beta: {:.3f} | mu: {:.3f} | mean: {:.3f}'
                  .format(beta, muS, mean))
        res = pd.concat((res, pd.DataFrame({'sym': sym, 'year': year, 'beta': beta,
                                       'mu': muS, 'mean': mean}, index=[0])))

In [ ]:
res[['mu', 'mean']].plot(kind='scatter', x='mu',
                         y='mean', figsize=(10, 6))
x = np.linspace(res['mu'].min(), res['mu'].max())
r = np.polyfit(res['mu'], res['mean'], deg=1)
plt.plot(x, x, 'r', label='identity')
plt.plot(x, np.polyval(r, x), 'g--', label='regression')
plt.legend();

In [ ]:
res[['beta', 'mean']].plot(kind='scatter', x='beta',
                         y='mean', figsize=(10, 6))
x = np.linspace(res['beta'].min(), res['beta'].max())
r = np.polyfit(res['beta'], res['mean'], deg=1)
plt.plot(x, np.polyval(r, x), 'g--', label='regression')
plt.legend();

## Multi-Factor Models

Ross (1976) introduces as follows:

> The purpose of this paper is to examine rigorously the arbitrage model of capital asset pricing developed in Ross (1971). The arbitrage model was proposed as an alternative to the mean variance capital asset pricing model, introduced by Sharpe, Lintner, and Treynor, that has become the major analytic tool for explaining phenomena observed in capital markets for risky assets.

In [ ]:
raw.columns

In [ ]:
sym = 'INTC.O'

In [ ]:
market = ['.SPX', '.VIX', 'EUR=', 'XAU=']

In [ ]:
reg = np.linalg.lstsq(rets[market], rets[sym], rcond=-1)[0]

In [ ]:
np.dot(rets[market].mean() * 252, reg)

In [ ]:
rets[sym].mean() * 252

### Same Year

In [ ]:
pr = False

In [ ]:
res = pd.DataFrame()
years = range(2010, 2019)
for sym in raw.columns[:5]:
    for year in years:
        sel = rets[(rets.index >= '01-01-{}'.format(year)) &
                  (rets.index <= '31-12-{}'.format(year))]
        
        beta = np.linalg.lstsq(sel[market],
                          sel[sym], rcond=-1)[0]
        muM = sel[market].mean() * 252
        muS = np.dot(muM, beta)
        mean =  sel[sym].mean() * 252
        if pr:
            print('\n' + sym + '| {}'.format(year))
            print(45 * '=')
            print('beta: {:.3} | mu: {:.3f} | mean: {:.3f}'
                  .format(beta, muS, mean))
        res = pd.concat((res, pd.DataFrame({'sym': sym, 'year': year, 'beta': beta.sum(),
                                       'mu': muS, 'mean': mean}, index=[0])))

In [ ]:
res[res['sym'] == 'MSFT.O'].set_index('year')[['mu', 'mean']].plot(kind='bar', figsize=(10, 6));

In [ ]:
res[['mu', 'mean']].plot(kind='scatter', x='mu',
                         y='mean', figsize=(10, 6))
x = np.linspace(res['mu'].min(), res['mu'].max())
r = np.polyfit(res['mu'], res['mean'], deg=1)
plt.plot(x, x, 'r', label='identity')
plt.plot(x, np.polyval(r, x), 'g--', label='regression')
plt.legend();

### Forward Looking

In [ ]:
res = pd.DataFrame()
years = range(2010, 2019)
for sym in raw.columns[:5]:
    for year in years:
        sel = rets[(rets.index >= '01-01-{}'.format(year)) &
                  (rets.index <= '31-12-{}'.format(year))]
        sel_ = rets[(rets.index >= '01-01-{}'.format(year+1)) &
                  (rets.index <= '31-12-{}'.format(year+1))]
        
        beta = np.linalg.lstsq(sel[market],
                          sel[sym], rcond=-1)[0]
        muM = sel_[market].mean() * 252
        muS = np.dot(muM, beta)
        mean =  sel_[sym].mean() * 252
        if pr:
            print('\n' + sym + '| {}'.format(year))
            print(45 * '=')
            print('beta: {:.3} | mu: {:.3f} | mean: {:.3f}'
                  .format(beta, muS, mean))
        res = pd.concat((res, pd.DataFrame({'sym': sym, 'year': year, 'beta': beta.sum(),
                                       'mu': muS, 'mean': mean}, index=[0])))

In [ ]:
res[res['sym'] == 'MSFT.O'].set_index('year')[['mu', 'mean']].plot(kind='bar', figsize=(10, 6));

In [ ]:
res.dropna(inplace=True)

In [ ]:
res[['mu', 'mean']].plot(kind='scatter', x='mu',
                         y='mean', figsize=(10, 6))
x = np.linspace(res['mu'].min(), res['mu'].max())
r = np.polyfit(res['mu'], res['mean'], deg=1)
plt.plot(x, x, 'r', label='identity')
plt.plot(x, np.polyval(r, x), 'g--', label='regression')
plt.legend();

## Principal Component Analysis (PCA)

Jolliffe and Cadima (2016) summarize:

> Large datasets are increasingly common and are often difficult to interpret. Principal component analysis (PCA) is a technique for reducing the dimensionality of such datasets, increasing interpretability but at the same time minimizing information loss. It does so by creating new uncorrelated variables that successively maximize variance.

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(3)

In [ ]:
pca.fit_transform(rets.iloc[:, 1:])

In [ ]:
pca.components_

In [ ]:
plt.bar(range(pca.n_components), pca.explained_variance_ratio_);

In [ ]:
pc = pd.DataFrame()
for i, w in enumerate(pca.components_):
    pc['pc_{}'.format(i)] = np.dot(rets.iloc[:, 1:], w)
pc.index = rets.index

In [ ]:
pc.head()

In [ ]:
pc.corr()

In [ ]:
res = pd.DataFrame()
years = range(2010, 2019)
for sym in raw.columns[:5]:
    for year in years:
        sel = rets[(rets.index >= '01-01-{}'.format(year)) &
                  (rets.index <= '31-12-{}'.format(year))]
        sel_ = pc[(pc.index >= '01-01-{}'.format(year)) &
                  (pc.index <= '31-12-{}'.format(year))]
        beta = np.linalg.lstsq(sel_,
                          sel[sym], rcond=-1)[0]
        muM = sel_.mean() * 252
        muS = np.dot(muM, beta)
        mean = sel[sym].mean() * 252
        if pr:
            print('\n' + sym + '| {}'.format(year))
            print(45 * '=')
            print('beta: {:.3} | mu: {:.3f} | mean: {:.3f}'
                  .format(beta, muS, mean))
        res = pd.concat((res, pd.DataFrame({'sym': sym, 'year': year, 'beta': beta.sum(),
                                       'mu': muS, 'mean': mean}, index=[0])))

In [ ]:
res[res['sym'] == 'AMZN.O'].set_index('year')[['mu', 'mean']].plot(kind='bar', figsize=(10, 6));

In [ ]:
res[['mu', 'mean']].plot(kind='scatter', x='mu',
                         y='mean', figsize=(10, 6))
x = np.linspace(res['mu'].min(), res['mu'].max())
r = np.polyfit(res['mu'], res['mean'], deg=1)
plt.plot(x, x, 'r', label='identity')
plt.plot(x, np.polyval(r, x), 'g--', label='regression')
plt.legend();

## Bibliography

* Markowitz, Harry (1952): &ldquo;Portfolio Selection.&rdquo; _Journal of Finance_, Vol. 7, No. 1, 77-91.
* Markowitz, Harry (1959): _Portfolio Selection._ John Wiley & Sons, New York.
* Ross, Stephen (1976): &ldquo;The Arbitrage Theory of Capital Asset Pricing.&rdquo; _Journal of Economic Theory_, Vol 13, 341-360.
* Sharpe, William (1964): &ldquo;Capital Asset Prices: A Theory of Market Equibrium under Conditions of Risk&rdquo; _Journal of Finance_, Vol. 19, No. 3, 425-442.
* Varian, Hal (2010): _Intermediate Microeconomics._ W.W.Norton & Company, New York & London.
* Jolliffe, Ian and Jorge Cadima (2006): &ldquo;Principal Component Analysis: A Review and Recent Developments.&rdquo; Philosophical Transactions, Vol. 374, 1-16.

<img src='http://hilpisch.com/taim_logo.png' width="350px" align="right">